<a href="https://colab.research.google.com/github/pokjay/heb-squad/blob/main/preprocess_translate_squad_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import pandas as pd

## Convert SQuAD JSON file to Pandas DataFrame

In [ ]:
!git clone https://github.com/pokjay/heb-squad

Cloning into 'heb-squad'...
remote: Enumerating objects: 64, done.
remote: Counting objects: 100% (64/64), done.
remote: Compressing objects: 100% (57/57), done.
remote: Total 64 (delta 19), reused 36 (delta 5), pack-reused 0
Unpacking objects: 100% (64/64), done.
Checking out files: 100% (21/21), done.


In [ ]:
with open('/content/heb-squad/data/original/dev-v2.0.json') as f_handle:
    dataset = json.load(f_handle)

In [ ]:
def handle_answer(article, context, qa, answer_field):

  rows = []

  for answer in qa[answer_field]:

    q = qa['question']
    qid = qa['id']
    is_impossible = qa['is_impossible']
    answer_end = answer['answer_start'] + len(answer['text'])
                
    context_marked = context[:answer['answer_start']] + \
                    '[0123] ' + \
                    context[answer['answer_start'] : answer_end] + \
                    ' [4567]' \
                    + context[answer_end:]
                
    row = {'article': article['title'],
           'id': qid,
           'context': context_marked,
           'question': q,
           'answer': answer['text'],
           'answer_start': answer['answer_start'],
           'answer_end': answer_end,
           'is_impossible': int(is_impossible)}

    rows.append(row)
  
  return rows

In [ ]:
rows = []

for article in dataset['data']:
  for p in article['paragraphs']:
    # Some lines have \n, we remove them as they screw up translation
    context = p['context'].replace('\n', '')
    
    for qa in p['qas']:
      if 'answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'answers'))
      if 'plausible_answers' in qa:
        rows.extend(handle_answer(article, context, qa, 'plausible_answers'))
      


In [ ]:
df = pd.DataFrame(rows).fillna("")
df.index = df.index.rename('idx')
df.shape

(26232, 8)

In [ ]:
df.head()

,article,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,,
0,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
1,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
2,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
3,Normans,56ddde6b9a695914005b9628,The Normans (Norman: Nourmands; French: Norman...,In what country is Normandy located?,France,159,165,0
4,Normans,56ddde6b9a695914005b9629,The Normans (Norman: Nourmands; French: Norman...,When were the Normans in Normandy?,10th and 11th centuries,94,117,0


In [ ]:
# df.drop(columns=['article']).to_excel('/content/heb-squad/data/intermediate/dev-v2.0.xlsx', index=True)

## 1. Translate XLSX manually using Google Translate Document translator
## 2. Run code below processing the translated output

In [ ]:
df2 = pd.read_csv('/content/heb-squad/data/intermediate/heb-dev-v2.0.tsv', sep='\t', quoting=3, index_col=0)

In [ ]:
df2['id'] = df2.id.str.strip()
df2['context'] = df2.context.str.strip()
df2['question'] = df2.question.str.strip()
df2['answer'] = df2.answer.str.strip()

Take a quick look at what we got

In [ ]:
df2.shape

(26232, 7)

In [ ]:
df2.head(3)

,id,context,question,answer,answer_start,answer_end,is_impossible
idx,,,,,,,
0,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0
1,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0
2,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0


### Validate IDs are correct, fix if needed

Check how many diffs we have

In [ ]:
pd.Series(df.id.values == df2.id.values).value_counts()

True     26152
False       80
dtype: int64

Print problematic IDs

In [ ]:
df2.loc[~(df.id.values == df2.id.values), 'id']

idx
34                                 28
528       תשלום 56e19557e3433e1400422
529       תשלום 56e19557e3433e1400422
530       תשלום 56e19557e3433e1400422
915          56e1cbe2cd28a01900c67 רע
                     ...             
25405           5737534ec3c5551400e51
25406           5737534ec3c5551400e51
25407           5737534ec3c5551400e51
25408           5737534ec3c5551400e51
25409           5737534ec3c5551400e51
Name: id, Length: 80, dtype: object

Print original IDs

In [ ]:
df.loc[~(df.id.values == df2.id.values), 'id']

idx
34       5ad3a266604f3c001a3fea28
528      56e19557e3433e1400422fee
529      56e19557e3433e1400422fee
530      56e19557e3433e1400422fee
915      56e1cbe2cd28a01900c67bad
                   ...           
25405    5737534ec3c5551400e51ead
25406    5737534ec3c5551400e51ead
25407    5737534ec3c5551400e51ead
25408    5737534ec3c5551400e51ead
25409    5737534ec3c5551400e51ead
Name: id, Length: 80, dtype: object

Replace problematic ids with original ids

In [ ]:
df2.loc[~(df.id.values == df2.id.values), 'id'] = df.loc[~(df.id.values == df2.id.values), 'id'].to_list()

In [ ]:
pd.Series(df.id.values == df2.id.values).value_counts()

True    26232
dtype: int64

### Add name of article for creating JSON later

In [ ]:
df2['article'] = df.article.to_list()
df2.article.value_counts() == df.article.value_counts()

Oxygen                                       True
Rhine                                        True
Economic_inequality                          True
Civil_disobedience                           True
Imperialism                                  True
Force                                        True
French_and_Indian_War                        True
European_Union_law                           True
Warsaw                                       True
Steam_engine                                 True
Immune_system                                True
Yuan_dynasty                                 True
Huguenot                                     True
Prime_number                                 True
Computational_complexity_theory              True
Islamism                                     True
University_of_Chicago                        True
Scottish_Parliament                          True
Southern_California                          True
Ctenophora                                   True


Remove markers from context, and then print a sample to see if context looks good

In [ ]:
df2['context_marked'] = df2.context

In [ ]:
df2['context'] = df2.context_marked.str.replace('[0123] ', '', regex=False)
df2['context'] = df2.context.str.replace(' [4567]', '', regex=False)

In [ ]:
df2.iloc[10].context_marked, '', '', df2.iloc[10].context

('הנורמנים (נורמן: נורמנדים; בצרפתית: נורמנדים; בלטינית: Normanni) היו האנשים שבמאות ה -10 וה -11 נתנו את שמם לנורמנדי, אזור בצרפת. הם צאצאים מפושעים ופיראטים נורדים ("נורמן" מגיע מ"נורסמן ") ודנאטים מ [0123] דנמרק, איסלנד ונורווגיה [4567], שתחת מנהיגם רולו הסכימו להישבע לנאמנות למלך צ\'ארלס השלישי ממערב פרנסיה. באמצעות דורות של התבוללות והתערבבות עם האוכלוסייה הפרנקאית והרומית-גאלית הילידית, צאצאיהם היו מתמזגים בהדרגה עם התרבויות הקרולינגיות של מערב פרנסיה המערבית. הזהות התרבותית והאתנית המובהקת של הנורמנים התפתחה בתחילה במחצית הראשונה של המאה ה -10, והיא המשיכה להתפתח במהלך המאות הבאות.',
 '',
 '',
 'הנורמנים (נורמן: נורמנדים; בצרפתית: נורמנדים; בלטינית: Normanni) היו האנשים שבמאות ה -10 וה -11 נתנו את שמם לנורמנדי, אזור בצרפת. הם צאצאים מפושעים ופיראטים נורדים ("נורמן" מגיע מ"נורסמן ") ודנאטים מ דנמרק, איסלנד ונורווגיה, שתחת מנהיגם רולו הסכימו להישבע לנאמנות למלך צ\'ארלס השלישי ממערב פרנסיה. באמצעות דורות של התבוללות והתערבבות עם האוכלוסייה הפרנקאית והרומית-גאלית הילידית, צאצאיהם הי

Calculate the new start and end locations of the answer in the Hebrew context

In [ ]:
df2['answer_start_heb'] = df2.context_marked.str.find('[0123] ')
df2['answer_end_heb'] = df2.apply(lambda row: row.context_marked.find(' [4567]', int(row.answer_start_heb)) - len(' [4567]'), axis=1)

Validate how many rows are problematic and will be discarded

In [ ]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     22566
False     3666
Name: answer_start_heb, dtype: int64

True     22167
False     4065
Name: answer_end_heb, dtype: int64


In [ ]:
for row in df2.sample(30):
  df2.iloc[10].context[int(df2.iloc[10].answer_start_heb):int(df2.iloc[10].answer_end_heb)]

In [ ]:
for row in df2.sample(10).itertuples():
  print(row.question)
  print(row.context[int(row.answer_start_heb):int(row.answer_end_heb)])
  print()

מתי מת אוגוסטוס?
לספירה 14

מתי השתלטה צרפת על לונדון?
1830

באילו אזורים בדרך כלל נמצאים חלקיקי משקעים?
מינים של חופים

מה מכסה מוזיאון פוג לאמנות?
אמנות מערבית מימי הביניים ועד היום

מה כוללת התבנית להצעות חוק שהעביר הפרלמנט הסקוטי?
[תאריך]

מהו הרווח הפוטנציאלי לעבודה שבה מעט עובדים מיומנים אך תפקידים רבים פנויים?
שכר גבוה

מי ניהל את Datanet 1 עבור KPN?
מחלקה אחת ב- KPN

אחת האסטרטגיות של האיסלאמיזציה היא לא לתפוס את השלטון באילו שיטות?


איזה ארכיאולוג ידוע האמין כי לאמזונס אין תושבים רבים?
בטי מגגרס

באיזה אמצעים אימפריאליזם לעולם אינו מנוהל?
בכוח צבאי



#### Try and fix answers ruined in translation
These are rows where the tags surrounding the answer was lost in translation, maeaning we cannot retrieve the correct indices of the answers from the Hebrew translation. We will try to use the translated answer to find the indices in the context.

In [ ]:
mis_ans_df = df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)]

In [ ]:
mis_ans_df.shape

(5302, 11)

In [ ]:
new_ans_start = mis_ans_df.apply(lambda row: row.context.find(row.answer), axis=1)
new_ans_end = mis_ans_df.apply(lambda row: -1 if (row.context.find(row.answer) == -1) else (row.context.find(row.answer) + len(row.answer)), axis=1)

new_ans_start_success = new_ans_start[new_ans_start > -1]
new_ans_end_success = new_ans_end[new_ans_end > -1]

df2.loc[new_ans_start_success.index, 'answer_start_heb'] = new_ans_start_success.to_list()
df2.loc[new_ans_end.index, 'answer_end_heb'] = new_ans_end.to_list()

In [ ]:
print((df2.answer_start_heb >= 0).value_counts())
print()
print((df2.answer_end_heb >= 0).value_counts())

True     24273
False     1959
Name: answer_start_heb, dtype: int64

True     23307
False     2925
Name: answer_end_heb, dtype: int64


Below we remove rows where the translation deleted the answer markers from the context.

In [ ]:
print('We will remove', df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)].shape[0], 'rows')

We will remove 2925 rows


In [ ]:
print('We will remove', df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)].id.nunique(), 'ids out of', df2.id.nunique())

We will remove 1654 ids out of 11858


In [ ]:
print('We will remove', df2[(df2.answer_start_heb >= df2.answer_end_heb) | (df2.answer_start_heb < 0) | (df2.answer_end_heb < 0)].id.nunique() / df2.id.nunique() * 100, 'of ids')

We will remove 13.948389273064599 of ids


In [ ]:
df2_clean = df2[(df2.answer_start_heb < df2.answer_end_heb) | (df2.answer_start_heb >= 0) | (df2.answer_end_heb >= 0)]

In [ ]:
df2_clean.shape

(24273, 11)

Now we can save the df to file, it is ready!

In [ ]:
!mkdir /content/heb-squad/data/final

mkdir: cannot create directory ‘/content/heb-squad/data/final’: File exists


In [ ]:
df2_clean.to_csv('/content/heb-squad/data/final/heb-dev-v2.0.csv', index=False)

*italicized text*## Output as JSON in SQuAD format for easy training and evaluation using SQuAD scripts

In [ ]:
!gzip -d /content/heb-squad/data/final/heb-dev-v2.0.csv.gz

gzip: /content/heb-squad/data/final/heb-dev-v2.0.csv already exists; do you wish to overwrite (y or n)? 
	not overwritten


In [ ]:
df = pd.read_csv('/content/heb-squad/data/final/heb-dev-v2.0.csv')

In [ ]:
df.head()

,id,context,question,answer,answer_start,answer_end,is_impossible,article,context_marked,answer_start_heb,answer_end_heb
0,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0,Normans,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,127,-1
1,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0,Normans,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,127,-1
2,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0,Normans,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,127,-1
3,56ddde6b9a695914005b9628,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,באיזו מדינה נמצאת נורמנדי?,צָרְפַת,159,165,0,Normans,הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; ל...,127,-1
4,56ddde6b9a695914005b9629,הנורמנים (נורמן: נורמנדס; צרפתית: נורמנדים; לט...,מתי היו הנורמנים בנורמנדי?,המאות ה -10 וה -11,94,117,0,Normans,הנורמנים (נורמן: נורמנדס; צרפתית: נורמנדים; לט...,99,-1


In [ ]:
for row in df.itertuples():
  print(row)
  break

Pandas(Index=0, id='56ddde6b9a695914005b9628', context='הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; לטינית: Normanni) היו האנשים שבמאות ה -10 וה -11 נתנו את שמם לנורמנדי, אזור בצרפת. הם צאצאים מהנורדים ("נורמן" מגיע מ"נורסמן ") פשיטות ושודדי ים מדנמרק, איסלנד ונורווגיה, שתחת מנהיגם רולו הסכימו להישבע לנאמנות למלך צ\'ארלס השלישי של מערב פרנסיה. באמצעות דורות של התבוללות והתערבבות עם האוכלוסייה הפרנקאית והרומית-גאלית הילידית, צאצאיהם היו מתמזגים בהדרגה עם התרבויות הקרולינגיות של מערב פרנסיה המערבית. הזהות התרבותית והאתנית המובהקת של הנורמנים התפתחה בתחילה במחצית הראשונה של המאה ה -10, והיא המשיכה להתפתח במהלך המאות הבאות.', question='באיזו מדינה נמצאת נורמנדי?', answer='צָרְפַת', answer_start=159, answer_end=165, is_impossible=0, article='Normans', context_marked='הנורמנים (נורמן: נורמנדים; צרפתית: נורמנדים; לטינית: Normanni) היו האנשים שבמאות ה -10 וה -11 נתנו את שמם לנורמנדי, אזור בצרפת [0123] [4567]. הם צאצאים מהנורדים ("נורמן" מגיע מ"נורסמן ") פשיטות ושודדי ים מדנמרק, איסלנד ונורווג

In [ ]:
articles = list(df.article.unique())

In [ ]:
squad_dataset = {'version': 'v2.0', 'data': []}

for article in articles:
  article_dict = {'title': article, 'paragraphs': []}
  squad_dataset['data'].append(article_dict)
  article_df = df[df.article == article]
  paragraphs = list(article_df.context.unique())

  for p in paragraphs:
    paragraph_dict = {'context': p, 'qas': []}
    article_dict['paragraphs'].append(paragraph_dict)
    p_df = article_df[article_df.context == p]
    questions = list(p_df.question.unique())

    for q in questions:
      
      q_df = p_df[p_df.question == q]
      
      # Squad saves plausible answers of impossible questions differently
      if bool(q_df.iloc[0].is_impossible) is False:
        answer_field = 'answers'
      else:
        answer_field = 'plausible_answers'
      
      q_dict = {answer_field: []}
      
      paragraph_dict['qas'].append(q_dict)
      
      # All rows have same id and is_impssible, so take first
      # Transform is_impossible to boolean as we saved it as integer 
      q_dict['id'] = q_df.iloc[0].id
      q_dict['is_impossible'] = bool(q_df.iloc[0].is_impossible)
      q_dict['question'] = q

      # Only these values are saved in SQuAD data
      ans_df = q_df[['answer', 'answer_start_heb']]

      for idx, row in ans_df.iterrows():
        answer_text, answer_start = row
        ans_dict = {'text': answer_text, 'answer_start': answer_start}
        q_dict[answer_field].append(ans_dict)


In [ ]:
with open('/content/heb-squad/data/final/heb-dev-v2.0.json', 'w', encoding='utf8') as f_handle:
    json.dump(squad_dataset, f_handle, ensure_ascii=False)